### Jesus Olivera
#### AIM 5001 Week 10 Assignment
#### Tidying and Transforming Data

#### Approach:

I created the csv file in excel making sure that the format was like that of the one provided in the HW. After, I uploaded the csv file into Github so I could extract the raw version and import it to this Jupyter Notebook. I used pandas to read the file and saved the data in variable called "df".
<br>


In [2]:
# Importing numpy, pandas and regex
import regex as re
import numpy as np
import pandas as pd

# The raw link is stored in variable called "filename" 
#I used "pandas.read_csv" to read the file and stored it in variable "data" 
filename = 'https://raw.githubusercontent.com/jaynuel/AIM-5001/master/AIM%20501%20Week%2010%20Assignment.csv'
data = pd.read_csv(filename)

# Creating a dataframe from variable "data"
df = pd.DataFrame(data)

# Variable called "df" contains the dataframe
df

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221,212.0,503.0,"1,841"
1,NaN,delayed,62.0,12,20.0,102.0,305
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AM WEST,on time,694.0,"4,840",383.0,320.0,201
4,NaN,delayed,117.0,415,65.0,129.0,61


### Tidyin and Transforming Data

Before transforming the data I used the pandas function "isnull()" and "sum()" to find and count the number of null values in the dataframe. I performed an iteration, contained in variable "rows_with_null", to find in which rows where the null values contained. If a row only contained null values the row was droped. I used the pandas function "dropna" to accomplished that.
<br>

**As shown below, there are 9 null values in the dataframe and they are located in the rows 1,2 and 4. Row 2 only contained null values so it was dropped.**

In [3]:
# Finding and counting the total null values
df.isnull().sum().sum()

9

In [4]:
# Iterating to find the rows containing null values
rows_with_null = [i for i, r in df.iterrows() if r.isnull().any()]
rows_with_null

[1, 2, 4]

In [5]:
# Row two was dropped due to only containing null values
df1 = df.dropna(how='all')
df1

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221,212.0,503.0,"1,841"
1,NaN,delayed,62.0,12,20.0,102.0,305
3,AM WEST,on time,694.0,"4,840",383.0,320.0,201
4,NaN,delayed,117.0,415,65.0,129.0,61


In order to change the format of the dataframe from "wide" to "long" I started by transposing the dataframe and by changing the column levels to index using the pandas function "stack()".The variable called "df1_transposed" contains the transposed dataframe and the variable called "stacked" contains the stacked dataframe.

In [6]:
# Transposing the dataframe
df1_transposed = df1.T
df1_transposed

,0,1,3,4
Unnamed: 0,ALASKA,NaN,AM WEST,NaN
Unnamed: 1,on time,delayed,on time,delayed
Los Angeles,497,62,694,117
Phoenix,221,12,"4,840",415
San Diego,212,20,383,65
San Francisco,503,102,320,129
Seattle,"1,841",305,201,61


In [7]:
# Using the stack function to change the column level to the index level.
stacked = df1.stack()
stacked

0  Unnamed: 0        ALASKA
   Unnamed: 1       on time
   Los Angeles          497
   Phoenix              221
   San Diego            212
   San Francisco        503
   Seattle            1,841
1  Unnamed: 1       delayed
   Los Angeles           62
   Phoenix               12
   San Diego             20
   San Francisco        102
   Seattle              305
3  Unnamed: 0       AM WEST
   Unnamed: 1       on time
   Los Angeles          694
   Phoenix            4,840
   San Diego            383
   San Francisco        320
   Seattle              201
4  Unnamed: 1       delayed
   Los Angeles          117
   Phoenix              415
   San Diego             65
   San Francisco        129
   Seattle               61
dtype: object

**I created dataframes indexing through the stacked dataframe.** 
* Variable "alk_on_time" contains values for the Alaska Airlines on-time flights. 
* Variable "alk_delay" contains values for the Alaska Airlines delay flights. 
* Variable "am_west_on_time" contains the values for the American West Airlines on-time flights. 
* Variable "am_west_delay" contains the values for the American West Airlines delay flights.
* Variable "airlines" contains the airline names
* Variable "destination" contains the names of the flight's destinations.

In [8]:
# Indexing dataframe to create separate dataframes for each on the values per airline
alk_on_time= pd.DataFrame(stacked[2:7])
alk_delay = pd.DataFrame(stacked[8:13])
am_west_on_time = pd.DataFrame(stacked[15:20])
am_west_delay= pd.DataFrame(stacked[21:26])

# Dataframe containing the airline values
ind = pd.Index(['Alaska','AM West'])
airlines = pd.DataFrame(ind.repeat(5))


# Dataframe containing flight's destinations
col = ['Los Angeles','Phoenix','San Diego','San Francisco','Seattle','Los Angeles','Phoenix','San Diego','San Francisco','Seattle']
destination = pd.DataFrame(col)


To update the dataframe to its long format I started by using the pandas "merge" function to merge all the data in its repective dataframe. I used the concat funtion to concatanate all the dataframes and preceded by updating the column names and replacing the commas from the colums containing them for nothing using regex so I could make the numeric values in the columns "on_time" and "delay" intergers. 

**Variables Description:**
* on_time = dataframe containing all on-time values
* delay = dataframe containing all delay values
* time = dataframe containing all values in a long format 

In [9]:
# Merging dataframes so all values corresponding to the same variable 
#are located in the same dataframe
on_time = pd.merge(alk_on_time, am_west_on_time, how='outer')
delay = pd.merge(alk_delay, am_west_delay, how='outer' )

# Concataninating dataframes
time= pd.concat([airlines,destination,on_time, delay], axis = 1, sort=True)

# Updating column titles 
time.columns =['airlines','destination','on_time','delay']

# Replacing ',' for nothing
time.replace(',','', regex=True, inplace=True)

# Transforming values in columns 'on_time' and 'delay' to intergers
time['on_time'] = time['on_time'].astype(int)
time['delay'] = time['delay'].astype(int)

# Dataframe in long format
time

,airlines,destination,on_time,delay
0,Alaska,Los Angeles,497,62
1,Alaska,Phoenix,221,12
2,Alaska,San Diego,212,20
3,Alaska,San Francisco,503,102
4,Alaska,Seattle,1841,305
5,AM West,Los Angeles,694,117
6,AM West,Phoenix,4840,415
7,AM West,San Diego,383,65
8,AM West,San Francisco,320,129
9,AM West,Seattle,201,61


### Data Analyzis

**Approach:**
<br>

To analyze which ariline has the best on-time performance by destination I added the total "on-time" flights for all Alaska and AM West airlines, groupiing by airline, and then divided each of the individual "on-time" flights for that number. I repeted the same process for the "delay" flights, and lastly divided the results obtained from the "on-time" division for the results obtained from the "delay" division, per airline,and per destination. The data has been normalized.    

In [10]:
# Adding all the Alaska "on-time" flights 
alaska_ontime = time[(time.airlines == "Alaska")]
alk_total_ontime = alaska_ontime['on_time'].sum()
# Adding all the Alaska "delay" flights
alaska_delay = time[(time.airlines == "Alaska")]
alk_total_delay = alaska_delay['delay'].sum()

# Adding all the AM West "on-time" flights
amwest_ontime = time[(time.airlines == "AM West")]
amwest_total_ontime = amwest_ontime['on_time'].sum()
# Adding all the AM West "delay" flights
amwest_delay = time[(time.airlines == "AM West")]
amwest_total_delay = amwest_delay['delay'].sum()


# Dividing each Alaska total, per destination, for the grand "on-time" total 
alaska_ontime_performance = alaska_ontime['on_time']/alk_total_ontime
# Dividing each Alaska total, per destination, for the grand "delay" total 
alaska_delay_performance = alaska_delay['delay']/alk_total_delay

# Divinding the Alaska "on-time" for the "delay" results per destination
alaska_performance = alaska_ontime_performance/alaska_delay_performance


# Dividing each AM West total, per destination, for the grand "on-time" total 
amwest_ontime_performance = amwest_ontime['on_time']/amwest_total_ontime
# Dividing each AM West total, per destination, for the grand "delay" total 
amwest_delay_performance = amwest_delay['delay']/amwest_total_delay

# Divinding the AM West "on-time" for the "delay" results per destination
amwest_performance = amwest_ontime_performance/amwest_delay_performance


# DataFrame containing Alaska performance results
alk_performance = pd.DataFrame(alaska_performance)
# DataFrame containing AM West performance results
amw_performance = pd.DataFrame(amwest_performance)


# DataFrame containing both, the Alaska and AM West on_time performance results
on_time_performance = pd.merge(alk_performance, amw_performance, how='outer')

# Print results
on_time_performance

,0
0,1.226659
1,2.818189
2,1.622053
3,0.754617
4,0.923662
5,0.725099
6,1.425677
7,0.720293
8,0.303238
9,0.402800


Variable called "on_time_performace" contains the analyzes results. To better visualize the results, I added them to the long format dataframe. 

In [11]:
# Adding the "on_time_performnace" column to the long format dataframe
time_performance= pd.concat([airlines,destination,on_time, delay,on_time_performance], axis = 1, sort=True)
#Adding the column titles
time_performance.columns =['airlines','destination','on_time','delay','on_time_performance']

# Results
time_performance.head(10).round(2)

,airlines,destination,on_time,delay,on_time_performance
0,Alaska,Los Angeles,497,62,1.23
1,Alaska,Phoenix,221,12,2.82
2,Alaska,San Diego,212,20,1.62
3,Alaska,San Francisco,503,102,0.75
4,Alaska,Seattle,"1,841",305,0.92
5,AM West,Los Angeles,694,117,0.73
6,AM West,Phoenix,"4,840",415,1.43
7,AM West,San Diego,383,65,0.72
8,AM West,San Francisco,320,129,0.30
9,AM West,Seattle,201,61,0.40


### Answers
**1. For each city, which airline had the best on time performance?**

The results above show that Alaska Airlines has better on-time performance for each of the destinations.

**2. Which airline had the best overall on time performance?**
 Alaska Airlines has the best overall 0n-time performance. The calculation below performs addition of all the on_time_performance, per airline. Results above show that Alaska Airlines has 7.34 overall chances of flights being on-time, versus AM West Airlines only having 3.58 overall chances of flights being on-time. 

In [12]:
# Adding the "on_time_performance" results grouping by airline.
time_performance.groupby('airlines').on_time_performance.sum().round(2)

airlines
AM West    3.58
Alaska     7.35
Name: on_time_performance, dtype: float64